In [88]:
import os
import requests as rq
import pandas as pd
import numpy as np
import json
import re
from bs4 import BeautifulSoup
from tqdm import tqdm,trange
from PIL import Image

In [49]:
Jump = [] # 只有第一次要執行這個，用來跳過前面已經查找過的

In [113]:
Start_ID = 5713
End_ID = 6300 #6300
root = 'https://www.vscinemas.com.tw/vsweb'  # get img url
path = r"C:\Users\User\Desktop\Hot_Movie_Img" # save in local 所以斜線方向不一樣

Type = {
    'movieTypeId':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    'movieTypeName':['奇幻','冒險','動作','劇情','犯罪','懸疑',
                     '驚悚','歷史','傳記','恐怖','愛情','動畫',
                     '推理','溫馨','家庭','喜劇','暴力','勵志',
                     '紀錄','音樂']
}

Type_CSV = pd.DataFrame(Type)
Type_CSV.to_csv("movieType.csv",index=False,encoding="utf_8_sig")

Grade = {
    'gradeId':[1,2,3,4,5],
    'gradeName':['general','childview','bigchild','teenager','adult']
    # 普0+  護6+  輔12+  輔15+  限18+     
}

Grade_CSV = pd.DataFrame(Grade)
Grade_CSV.to_csv("grade.csv",index=False,encoding="utf_8_sig")



Movie_Info = {
    'movieId':[], # int 255
    'movieName':[], # CH Name string 30
    'movieStart':[],# string 11
    'gradeId':[], # gradeId eg 分級
    'director':[],'actors':[],'movieTypeId':[], # typeId eg 愛情 熱血 動作...
    'movieTime':[],# 片長  
    'movieImg':[], # url_type
    'movieInfo':[],
    'moreMovieImg':[],
    'moreMovieImgNum':[]
}


In [114]:
for i in tqdm(range(Start_ID,End_ID)):   # tqdm 可以比較清楚現在進度
    if i in Jump:
        continue
    resp = rq.get(
        f"https://www.vscinemas.com.tw/vsweb/film/detail.aspx?id={i}"
    )
    content = BeautifulSoup(resp.text, "html.parser")
    try:
        ## movieTime  這个先判斷，如果為空，代表已下架
        Times = content.select('div.movieTime article div.movieDay h4')  # 去對應tag找內容 有另一種寫法是像下面醬
        MoviePlayTime = set()                                            # Times = content.find_all(div,{"class":movieTime})
        for t in Times:
            MoviePlayTime.add(t.text)
        if not MoviePlayTime: # 表示近期沒有撥放 以下架
            Jump.append(i)
            continue
        ## titleArea movie name, time 
        movieName = content.select_one('div.titleArea h1').text # 電影名稱 中文
        movieName = re.sub(r'[ ．–·，〜!‧:：　-]|『.*',"",movieName) # 正則表達 去掉全部特殊符
        movieName = movieName.upper()
        # 全形轉半形
        tmpName = movieName
        movieName=""
        for s in tmpName:
            ch_num = ord(s)
            if(ch_num>=65281 and ch_num<=65347):
                ch_num -=65248
            movieName += chr(ch_num)
        #EN_Name = content.select_one('div.titleArea h2').text
        movieStart = content.select_one('div.titleArea time').text.replace("上映日期：","") # 上映日期

        ## markArea Level
        grade = content.select('div.markArea span')[0].get('class') # eg 保護級 是 list
        gradeId = Grade['gradeId'][Grade['gradeName'].index(f'{grade[0]}')]
        #Mark2 = content.select('div.markArea span')[1].get('class') # eg 熱售
        
        ## infoArea Director, Actor, Type, L_of_File
        inforArea =content.select_one('div.infoArea table')
        if inforArea != None:  # 直播類型沒有下面這些
            director = content.select_one('div.infoArea table tr td p').text # 導演
            actors = content.select('div.infoArea table tr')[1].select_one('td p').text   # 主演
            movieTime = content.select('div.infoArea table tr')[3].select('td')[1].text # 片長
            typeform = content.select('div.infoArea table tr')[2].select('td')[1].text[0:2] # 我只有選一個 也可以都拿~~
            if typeform != "": # 有的電影沒有類型
                movieTypeId = Type['movieTypeId'][Type['movieTypeName'].index(f'{typeform}')]
        ## Version
#         VersionList = content.select('div.movieVersion ul li a.versionFirst')
#         Version = []
#         for v in VersionList:
#             Version.append(v.text)   
        ## IMG
        Img = content.select_one('div.movieMain figure img').get('src').replace('..',root)
        Img = rq.get(Img)  # 拿到Img
        if not os.path.exists(path):
            os.mkdir(path)
        with open(f"{path}\_{movieName}_{i}.jpg","wb") as file:
            file.write(Img.content)
            Img = Image.open(f"{path}\_{movieName}_{i}.jpg")
            ReImg = Img.resize((500,740))  # 修正大小
            ReImg.save(f"{path}\_{movieName}_{i}.jpg")
            Img_URL = f"{path}\_{movieName}_{i}.jpg"
        ## movieStory
        movieInfo = content.select_one('div.movieStory p').text
        
        ## morePicture
        More_Picture = content.select('div.moviePicture a')
        moreImg = ""
        moreImgNum = 0
        if More_Picture:
            for img in More_Picture:
                moreImgNum += 1
                img_url = img.get('href').replace("..",root)
                img = rq.get(img_url)
                if not os.path.exists(path+f'\more_{movieName}_{i}'):  # 建立dir
                    os.mkdir(path+f'\more_{movieName}_{i}')
                with open(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg","wb") as file:
                    file.write(img.content)
                    Img = Image.open(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
                    ReImg = Img.resize((800,300))
                    ReImg.save(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
                    #moreImg.append(f"{path}\more_{movieName}_{i}\_{moreImgNum}.jpg")
            moreImg = path+f'\more_{movieName}_{i}'
        ########### save into dict 
#         for ver in Version:
        Movie_Info['movieId'].append(i)
        Movie_Info['movieName'].append(movieName)
        #Movie_Info['EN_Name'].append(EN_Name)
        Movie_Info['movieStart'].append(movieStart)
        Movie_Info['gradeId'].append(gradeId)
        #Movie_Info['Mark2'].append(Mark2)
        Movie_Info['director'].append(director)
        Movie_Info['actors'].append(actors)
        Movie_Info['movieTypeId'].append(movieTypeId)
        Movie_Info['movieTime'].append(movieTime)
        #Movie_Info['Version'].append(ver)
        Movie_Info['movieImg'].append(Img_URL)
        Movie_Info['movieInfo'].append(movieInfo)
        Movie_Info['moreMovieImg'].append(moreImg)
        Movie_Info['moreMovieImgNum'].append(moreImgNum)
    except:
        print("Error ID:",i)  # 

100%|████████████████████████████████████████████████████████████████████████████████| 587/587 [01:16<00:00,  7.70it/s]


In [53]:
Movie_CSV = pd.DataFrame(Movie_Info)
Movie_CSV.to_csv("movie.csv",index=False,encoding="utf_8_sig")

In [115]:
movie_json = {'movieId':Movie_Info['movieId'],'movieName':Movie_Info['movieName']} # for other jupyter load
with open("movie.json","wt") as f:
    json.dump(movie_json,f)   

In [ ]:
：
: